# Notebook 5 : Améliorations avancées

Ce notebook regroupe 3 pistes d'amélioration :

1. **Kalman avec modèle de vitesse constante** : au lieu de supposer que la souris est immobile, on modélise l'état comme `[x, y, vx, vy]` — le filtre anticipe le mouvement
2. **Ensemble de modèles** : on combine les prédictions XGBoost + Transformer (+ CNN) par moyenne pondérée ou stacking
3. **GRU multi-fenêtre** : on utilise les N fenêtres précédentes comme contexte temporel — un GRU au-dessus du Transformer apprend la continuité de la trajectoire

**Prérequis** : avoir exécuté les notebooks 1, 2, 3 (les fichiers `.npy` et `.pt` doivent exister).

## 1. Imports et configuration

In [ ]:
import pandas as pd
import numpy as np
import json
import os
import math
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.linear_model import Ridge

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

import warnings
warnings.filterwarnings('ignore')

SEED = 42
np.random.seed(SEED)
torch.manual_seed(SEED)

if torch.cuda.is_available():
    DEVICE = torch.device('cuda')
elif torch.backends.mps.is_available():
    DEVICE = torch.device('mps')
else:
    DEVICE = torch.device('cpu')
print(f'Device: {DEVICE}')

In [ ]:
# --- Connexion S3 (Onyxia) ou chargement local ---
S3_ENDPOINT = "https://minio.lab.sspcloud.fr"
S3_BUCKET = "gmarguier"
S3_PREFIX = "hacktion-potential"

PARQUET_NAME = "M1199_PAG_stride4_win108_test.parquet"
JSON_NAME = "M1199_PAG.json"

LOCAL_DIR = os.path.join(os.path.abspath('..'), 'data')
LOCAL_PARQUET = os.path.join(LOCAL_DIR, PARQUET_NAME)
LOCAL_JSON = os.path.join(LOCAL_DIR, JSON_NAME)

USE_S3 = False
fs = None

try:
    import s3fs
    fs = s3fs.S3FileSystem(
        client_kwargs={"endpoint_url": S3_ENDPOINT},
        key=os.environ.get("AWS_ACCESS_KEY_ID"),
        secret=os.environ.get("AWS_SECRET_ACCESS_KEY"),
        token=os.environ.get("AWS_SESSION_TOKEN"),
    )
    s3_parquet = f"{S3_BUCKET}/{S3_PREFIX}/{PARQUET_NAME}"
    if fs.exists(s3_parquet):
        USE_S3 = True
        print(f"[S3] Connexion OK — lecture depuis s3://{S3_BUCKET}/{S3_PREFIX}/")
    else:
        print(f"[S3] Bucket accessible mais fichier introuvable ({s3_parquet})")
except Exception as e:
    print(f"[S3] Non disponible ({type(e).__name__}: {e})")

if not USE_S3:
    if os.path.exists(LOCAL_PARQUET):
        print(f"[LOCAL] Chargement depuis {LOCAL_DIR}/")
    else:
        raise FileNotFoundError(f"Données introuvables ni sur S3 ni en local ({LOCAL_PARQUET})")

In [ ]:
def evaluate(name, y_true, y_pred):
    """Calcule et affiche les métriques."""
    r2_x = r2_score(y_true[:, 0], y_pred[:, 0])
    r2_y = r2_score(y_true[:, 1], y_pred[:, 1])
    eucl = np.sqrt((y_true[:, 0] - y_pred[:, 0])**2 + (y_true[:, 1] - y_pred[:, 1])**2)
    print(f'=== {name} ===')
    print(f'  R²   : X={r2_x:.4f}, Y={r2_y:.4f}')
    print(f'  Eucl : mean={eucl.mean():.4f}, median={np.median(eucl):.4f}, p90={np.percentile(eucl, 90):.4f}')
    return {'r2_x': r2_x, 'r2_y': r2_y, 'eucl': eucl}

---
# Partie 1 : Filtre de Kalman avec modèle de vitesse constante

## Pourquoi ?

Le Kalman du notebook 1-3 utilise un modèle de **position constante** : il suppose que la souris ne bouge pas entre deux observations. C'est une approximation très grossière — une souris en mouvement parcourt une distance significative entre deux fenêtres de 108ms.

Le modèle de **vitesse constante** utilise un état étendu `[x, y, vx, vy]` :
- **Prédiction** : `x(t+1) = x(t) + vx(t)*dt`, `vx(t+1) = vx(t)` (la vitesse persiste)
- **Mesure** : on observe `[x, y]` (les prédictions du modèle ML)

Cela permet au filtre d'**anticiper** le mouvement plutôt que de simplement lisser.

In [ ]:
def kalman_velocity(observations, process_noise_pos=0.001, process_noise_vel=0.01,
                     measurement_noise=0.01, dt=1.0):
    """
    Filtre de Kalman avec modèle de vitesse constante.
    
    État : [x, y, vx, vy]
    Transition : x += vx*dt, y += vy*dt, vx = vx, vy = vy
    Observation : [x, y] (prédictions du modèle ML)
    
    Args:
        observations: (N, 2) - prédictions brutes
        process_noise_pos: bruit sur la position (Q_pos)
        process_noise_vel: bruit sur la vitesse (Q_vel)
        measurement_noise: bruit de mesure (R)
        dt: pas de temps normalisé entre observations
    
    Returns:
        smoothed: (N, 2) - positions lissées
    """
    N = len(observations)
    smoothed = np.zeros((N, 2), dtype=np.float64)
    
    # Matrice de transition : vitesse constante
    F = np.array([
        [1, 0, dt, 0],
        [0, 1, 0, dt],
        [0, 0, 1,  0],
        [0, 0, 0,  1]
    ], dtype=np.float64)
    
    # Matrice d'observation : on observe seulement (x, y)
    H = np.array([
        [1, 0, 0, 0],
        [0, 1, 0, 0]
    ], dtype=np.float64)
    
    # Bruit du processus
    Q = np.diag([process_noise_pos, process_noise_pos,
                 process_noise_vel, process_noise_vel])
    
    # Bruit de mesure
    R = np.eye(2) * measurement_noise
    
    # État initial : position = première observation, vitesse = 0
    x_est = np.array([observations[0, 0], observations[0, 1], 0.0, 0.0], dtype=np.float64)
    P = np.diag([measurement_noise, measurement_noise, 0.1, 0.1])
    
    I4 = np.eye(4)
    
    for t in range(N):
        # --- Prédiction ---
        x_pred = F @ x_est
        P_pred = F @ P @ F.T + Q
        
        # --- Mise à jour ---
        z = observations[t].astype(np.float64)
        y_innov = z - H @ x_pred
        S = H @ P_pred @ H.T + R
        K = P_pred @ H.T @ np.linalg.inv(S)
        
        x_est = x_pred + K @ y_innov
        P = (I4 - K @ H) @ P_pred
        
        smoothed[t] = x_est[:2]  # on ne retourne que (x, y)
    
    return smoothed.astype(np.float32)

In [ ]:
# Charger les prédictions des notebooks précédents
preds = {}
y_tests = {}

for name, pred_file, true_file in [
    ('XGBoost', '../outputs/preds_xgboost.npy', '../outputs/y_test.npy'),
    ('Transformer', '../outputs/preds_transformer.npy', '../outputs/y_test_transformer.npy'),
    ('CNN', '../outputs/preds_cnn.npy', '../outputs/y_test_cnn.npy'),
]:
    if os.path.exists(pred_file) and os.path.exists(true_file):
        preds[name] = np.load(pred_file)
        y_tests[name] = np.load(true_file)
        print(f'{name}: {preds[name].shape}')

print(f'\nModèles disponibles : {list(preds.keys())}')

In [ ]:
# Ancien Kalman (position constante) pour comparaison
def kalman_position(observations, process_noise=0.001, measurement_noise=0.01):
    """Kalman avec modèle de position constante (comme dans les notebooks 1-3)."""
    N = len(observations)
    smoothed = np.zeros_like(observations)
    x_est = observations[0].copy()
    P = np.eye(2) * measurement_noise
    Q = np.eye(2) * process_noise
    R = np.eye(2) * measurement_noise
    for t in range(N):
        x_pred = x_est
        P_pred = P + Q
        z = observations[t]
        y_innov = z - x_pred
        S = P_pred + R
        K = P_pred @ np.linalg.inv(S)
        x_est = x_pred + K @ y_innov
        P = (np.eye(2) - K) @ P_pred
        smoothed[t] = x_est
    return smoothed

In [ ]:
# Tester le Kalman vitesse sur chaque modèle
# On compare : brut, Kalman position, Kalman vitesse

kalman_vel_configs = {
    'Vel léger': {'process_noise_pos': 0.005, 'process_noise_vel': 0.05, 'measurement_noise': 0.005},
    'Vel moyen': {'process_noise_pos': 0.001, 'process_noise_vel': 0.02, 'measurement_noise': 0.02},
    'Vel fort':  {'process_noise_pos': 0.0005, 'process_noise_vel': 0.01, 'measurement_noise': 0.05},
}

kalman_vel_results = {}  # {model_name: {config_name: y_smooth}}

for model_name in preds:
    y_pred = preds[model_name]
    y_true = y_tests[model_name]
    
    print(f'\n{"="*60}')
    print(f'Modèle : {model_name}')
    print(f'{"="*60}')
    
    # Brut
    evaluate(f'{model_name} brut', y_true, y_pred)
    
    # Kalman position (meilleur config des notebooks)
    y_pos = kalman_position(y_pred, process_noise=0.001, measurement_noise=0.02)
    evaluate(f'{model_name} + Kalman position', y_true, y_pos)
    
    # Kalman vitesse
    kalman_vel_results[model_name] = {}
    best_eucl = float('inf')
    best_config = None
    
    for config_name, params in kalman_vel_configs.items():
        y_vel = kalman_velocity(y_pred, **params)
        res = evaluate(f'{model_name} + Kalman {config_name}', y_true, y_vel)
        kalman_vel_results[model_name][config_name] = y_vel
        
        if res['eucl'].mean() < best_eucl:
            best_eucl = res['eucl'].mean()
            best_config = config_name
    
    print(f'\n  >> Meilleur Kalman vitesse : {best_config} (eucl={best_eucl:.4f})')
    
    # Sauvegarder le meilleur
    fname = f'../outputs/preds_{model_name.lower()}_kalman_vel.npy'
    np.save(fname, kalman_vel_results[model_name][best_config])
    print(f'  >> Sauvegardé : {fname}')

In [ ]:
# Visualisation : Kalman position vs Kalman vitesse (sur le Transformer)
if 'Transformer' in preds:
    y_pred_t = preds['Transformer']
    y_true_t = y_tests['Transformer']
    
    y_pos = kalman_position(y_pred_t, 0.001, 0.02)
    y_vel = kalman_velocity(y_pred_t, 0.001, 0.02, 0.02)
    
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    seg = slice(0, 300)
    
    # Trajectoire
    ax = axes[0, 0]
    ax.plot(y_true_t[seg, 0], y_true_t[seg, 1], 'b-', alpha=0.5, label='Vérité', linewidth=1.5)
    ax.plot(y_pred_t[seg, 0], y_pred_t[seg, 1], 'r-', alpha=0.3, label='Brut', linewidth=0.8)
    ax.plot(y_pos[seg, 0], y_pos[seg, 1], 'orange', alpha=0.6, label='Kalman position', linewidth=1.2)
    ax.plot(y_vel[seg, 0], y_vel[seg, 1], 'g-', alpha=0.7, label='Kalman vitesse', linewidth=1.5)
    ax.set_xlabel('X'); ax.set_ylabel('Y')
    ax.set_title('Trajectoire (300 points test)')
    ax.legend(); ax.set_aspect('equal')
    
    # Position X
    ax = axes[0, 1]
    t_idx = np.arange(200)
    ax.plot(t_idx, y_true_t[t_idx, 0], 'b-', label='Vérité', linewidth=2)
    ax.plot(t_idx, y_pred_t[t_idx, 0], 'r-', alpha=0.3, label='Brut', linewidth=0.8)
    ax.plot(t_idx, y_pos[t_idx, 0], 'orange', alpha=0.6, label='Kalman position', linewidth=1.2)
    ax.plot(t_idx, y_vel[t_idx, 0], 'g-', label='Kalman vitesse', linewidth=1.5)
    ax.set_xlabel('Index'); ax.set_ylabel('X')
    ax.set_title('Position X - Zoom 200 points')
    ax.legend()
    
    # CDF
    ax = axes[1, 0]
    for label, yp, color in [
        ('Brut', y_pred_t, 'red'),
        ('Kalman position', y_pos, 'orange'),
        ('Kalman vitesse', y_vel, 'green'),
    ]:
        e = np.sort(np.sqrt((y_true_t[:, 0] - yp[:, 0])**2 + (y_true_t[:, 1] - yp[:, 1])**2))
        ax.plot(e, np.linspace(0, 1, len(e)), label=label, linewidth=2, color=color)
    ax.set_xlabel('Erreur euclidienne'); ax.set_ylabel('CDF')
    ax.set_title('CDF des erreurs - Transformer')
    ax.legend(); ax.grid(True, alpha=0.3)
    
    # Barplot comparatif
    ax = axes[1, 1]
    eucl_brut = np.sqrt((y_true_t[:, 0] - y_pred_t[:, 0])**2 + (y_true_t[:, 1] - y_pred_t[:, 1])**2)
    eucl_pos = np.sqrt((y_true_t[:, 0] - y_pos[:, 0])**2 + (y_true_t[:, 1] - y_pos[:, 1])**2)
    eucl_vel = np.sqrt((y_true_t[:, 0] - y_vel[:, 0])**2 + (y_true_t[:, 1] - y_vel[:, 1])**2)
    
    names = ['Brut', 'Kalman\nposition', 'Kalman\nvitesse']
    means = [eucl_brut.mean(), eucl_pos.mean(), eucl_vel.mean()]
    p90s = [np.percentile(eucl_brut, 90), np.percentile(eucl_pos, 90), np.percentile(eucl_vel, 90)]
    
    x_pos = np.arange(3)
    ax.bar(x_pos - 0.15, means, 0.3, label='Moyenne', color='steelblue')
    ax.bar(x_pos + 0.15, p90s, 0.3, label='P90', color='coral')
    ax.set_xticks(x_pos); ax.set_xticklabels(names)
    ax.set_ylabel('Erreur euclidienne')
    ax.set_title('Comparaison Kalman - Transformer')
    ax.legend()
    
    plt.suptitle('Kalman position vs vitesse constante (Transformer)', fontsize=14, fontweight='bold', y=1.01)
    plt.tight_layout()
    plt.show()

---
# Partie 2 : Ensemble de modèles

## Pourquoi ?

Chaque modèle capture des aspects différents des données :
- **XGBoost** : features statistiques manuelles, robuste mais perd l'info fine
- **Transformer** : waveforms bruts, co-activations, mais peu de données d'entraînement
- **CNN** : patterns spatio-temporels, mais agrège les spikes en bins

En combinant leurs prédictions, on exploite leur complémentarité.

**3 stratégies :**
1. Moyenne simple
2. Moyenne pondérée (poids inversement proportionnels à l'erreur)
3. Stacking : un méta-modèle (Ridge) apprend la meilleure combinaison

In [ ]:
# Vérifier que tous les modèles ont le même test set
available_models = list(preds.keys())
print(f'Modèles disponibles : {available_models}')

# Vérifier la cohérence des y_test
ref_true = y_tests[available_models[0]]
for name in available_models[1:]:
    diff = np.abs(y_tests[name] - ref_true[:len(y_tests[name])]).max()
    print(f'  Diff max y_test {available_models[0]} vs {name}: {diff:.8f}')
    
# Utiliser le y_test de référence
y_true_ensemble = ref_true
n_test = len(y_true_ensemble)
print(f'\nTaille test set : {n_test}')

In [ ]:
# --- Stratégie 1 : Moyenne simple ---
pred_stack = np.stack([preds[name][:n_test] for name in available_models])  # (n_models, N, 2)
y_mean = pred_stack.mean(axis=0)
res_mean = evaluate('Ensemble moyenne simple', y_true_ensemble, y_mean)
print()

# --- Stratégie 2 : Moyenne pondérée (poids ∝ 1/erreur) ---
weights = []
for name in available_models:
    eucl = np.sqrt((y_true_ensemble[:, 0] - preds[name][:n_test, 0])**2 + 
                   (y_true_ensemble[:, 1] - preds[name][:n_test, 1])**2).mean()
    weights.append(1.0 / eucl)
    
weights = np.array(weights)
weights = weights / weights.sum()
print(f'Poids calculés : {dict(zip(available_models, [f"{w:.3f}" for w in weights]))}')

y_weighted = np.zeros_like(y_true_ensemble)
for i, name in enumerate(available_models):
    y_weighted += weights[i] * preds[name][:n_test]

res_weighted = evaluate('Ensemble moyenne pondérée', y_true_ensemble, y_weighted)
print()

# --- Stratégie 3 : Stacking avec Ridge ---
# On utilise la première moitié du test set pour entraîner le méta-modèle,
# et la seconde moitié pour évaluer.
# (Idéalement on utiliserait un val set séparé, mais on fait avec ce qu'on a)
half = n_test // 2

# Construire la matrice de features : concaténer les prédictions de chaque modèle
X_stack = np.concatenate([preds[name][:n_test] for name in available_models], axis=1)  # (N, 2*n_models)

X_stack_train = X_stack[:half]
y_stack_train = y_true_ensemble[:half]
X_stack_test = X_stack[half:]
y_stack_test = y_true_ensemble[half:]

ridge = Ridge(alpha=1.0)
ridge.fit(X_stack_train, y_stack_train)
y_stacked = ridge.predict(X_stack_test)

res_stacked = evaluate('Ensemble stacking (Ridge, 2e moitié test)', y_stack_test, y_stacked)

# Afficher les coefficients du méta-modèle
print(f'\nCoefficients Ridge :')
for i, name in enumerate(available_models):
    print(f'  {name}: coef_x={ridge.coef_[0, 2*i]:.3f}, coef_y={ridge.coef_[1, 2*i+1]:.3f}')

In [ ]:
# --- Stratégie bonus : Ensemble + Kalman vitesse ---
# On applique le meilleur Kalman vitesse sur la moyenne pondérée
y_ensemble_kalman = kalman_velocity(y_weighted, 
                                     process_noise_pos=0.001, 
                                     process_noise_vel=0.02,
                                     measurement_noise=0.02)
res_ensemble_kalman = evaluate('Ensemble pondéré + Kalman vitesse', y_true_ensemble, y_ensemble_kalman)

# Sauvegarder les prédictions ensemble
np.save('../outputs/preds_ensemble_weighted.npy', y_weighted)
np.save('../outputs/preds_ensemble_kalman_vel.npy', y_ensemble_kalman)
print('\nPrédictions ensemble sauvegardées.')

In [ ]:
# Visualisation de l'ensemble
fig, axes = plt.subplots(1, 3, figsize=(18, 5))
seg = slice(0, 300)

# Trajectoire
ax = axes[0]
ax.plot(y_true_ensemble[seg, 0], y_true_ensemble[seg, 1], 'b-', alpha=0.5, label='Vérité', linewidth=1.5)
for name in available_models:
    ax.plot(preds[name][seg, 0], preds[name][seg, 1], '-', alpha=0.2, linewidth=0.5, label=name)
ax.plot(y_weighted[seg, 0], y_weighted[seg, 1], 'orange', alpha=0.6, label='Ensemble pondéré', linewidth=1.2)
ax.plot(y_ensemble_kalman[seg, 0], y_ensemble_kalman[seg, 1], 'g-', label='Ensemble + Kalman vel', linewidth=1.5)
ax.set_xlabel('X'); ax.set_ylabel('Y')
ax.set_title('Trajectoires')
ax.legend(fontsize=8); ax.set_aspect('equal')

# CDF
ax = axes[1]
for name, yp, color in [
    *[(n, preds[n][:n_test], None) for n in available_models],
    ('Ensemble pondéré', y_weighted, 'orange'),
    ('Ensemble + Kalman vel', y_ensemble_kalman, 'green'),
]:
    e = np.sort(np.sqrt((y_true_ensemble[:, 0] - yp[:, 0])**2 + (y_true_ensemble[:, 1] - yp[:, 1])**2))
    kwargs = {'color': color} if color else {}
    lw = 2.5 if color else 1
    ax.plot(e, np.linspace(0, 1, len(e)), label=name, linewidth=lw, **kwargs)
ax.set_xlabel('Erreur euclidienne'); ax.set_ylabel('CDF')
ax.set_title('CDF des erreurs')
ax.legend(fontsize=8); ax.grid(True, alpha=0.3)

# Barplot
ax = axes[2]
all_names = available_models + ['Ensemble\npondéré', 'Ensemble\n+ Kalman vel']
all_preds_list = [preds[n][:n_test] for n in available_models] + [y_weighted, y_ensemble_kalman]
all_means = []
for yp in all_preds_list:
    e = np.sqrt((y_true_ensemble[:, 0] - yp[:, 0])**2 + (y_true_ensemble[:, 1] - yp[:, 1])**2)
    all_means.append(e.mean())

colors = ['#66c2a5', '#fc8d62', '#8da0cb', '#e78ac3', '#a6d854']
bars = ax.bar(range(len(all_names)), all_means, color=colors[:len(all_names)], edgecolor='black')
ax.set_xticks(range(len(all_names))); ax.set_xticklabels(all_names, fontsize=9)
ax.set_ylabel('Erreur euclidienne moyenne')
ax.set_title('Comparaison')
for bar, val in zip(bars, all_means):
    ax.text(bar.get_x() + bar.get_width()/2., bar.get_height() + 0.002,
            f'{val:.4f}', ha='center', va='bottom', fontsize=9)

plt.suptitle('Ensemble de modèles', fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

---
# Partie 3 : GRU multi-fenêtre (contexte temporel)

## Pourquoi ?

Tous les modèles précédents traitent chaque fenêtre de 108ms **indépendamment**. Or la position à l'instant t est fortement corrélée à la position à t-1, t-2, etc. Un GRU (Gated Recurrent Unit) peut apprendre cette continuité temporelle.

## Architecture

1. **Encodeur de fenêtre** : on réutilise le Transformer pré-entraîné du notebook 2 comme extracteur de features. Pour chaque fenêtre, il produit un vecteur de dimension D=64.
2. **GRU** : prend la séquence de K vecteurs consécutifs → apprend la dynamique temporelle
3. **Readout** : Dense → (x, y)

On freeze le Transformer (pas de fine-tuning) pour éviter l'overfitting, et on n'entraîne que le GRU + readout.

**Alternative plus légère** : au lieu de réutiliser le Transformer, on peut utiliser les features XGBoost (déjà calculées) comme entrée du GRU. C'est beaucoup plus rapide et ne nécessite pas de GPU.

### 3a. Chargement des données et extraction des features

In [ ]:
# Charger les données brutes
print('Chargement des données...')
if USE_S3:
    with fs.open(f"{S3_BUCKET}/{S3_PREFIX}/{PARQUET_NAME}", "rb") as f:
        df = pd.read_parquet(f)
    with fs.open(f"{S3_BUCKET}/{S3_PREFIX}/{JSON_NAME}", "r") as f:
        params = json.load(f)
else:
    df = pd.read_parquet(LOCAL_PARQUET)
    with open(LOCAL_JSON, 'r') as f:
        params = json.load(f)

nGroups = params['nGroups']
nChannelsPerGroup = [params[f'group{g}']['nChannels'] for g in range(nGroups)]

speed_masks = np.array([x[0] for x in df['speedMask']])
df_moving = df[speed_masks].reset_index(drop=True)
print(f'Exemples en mouvement : {len(df_moving)}')

In [ ]:
# Extraire les features (même code que notebook 1)
def extract_features(row, nGroups, nChannelsPerGroup):
    features = {}
    total_spikes = 0
    all_amplitudes = []
    
    for g in range(nGroups):
        nCh = nChannelsPerGroup[g]
        raw = row[f'group{g}']
        waveforms = raw.reshape(-1, nCh, 32)
        n_spikes = waveforms.shape[0]
        
        features[f'shank{g}_n_spikes'] = n_spikes
        total_spikes += n_spikes
        
        if n_spikes > 0:
            amplitudes = waveforms.max(axis=2) - waveforms.min(axis=2)  # (n_spikes, nCh)
            max_amp_per_spike = amplitudes.max(axis=1)  # (n_spikes,)
            all_amplitudes.extend(max_amp_per_spike)
            
            features[f'shank{g}_amp_mean'] = amplitudes.mean()
            features[f'shank{g}_amp_max'] = amplitudes.max()
            features[f'shank{g}_amp_std'] = amplitudes.std()
            features[f'shank{g}_energy'] = (waveforms ** 2).mean()
            features[f'shank{g}_dominant_ch'] = amplitudes.mean(axis=0).argmax()
            
            for ch in range(nCh):
                features[f'shank{g}_ch{ch}_amp'] = amplitudes[:, ch].mean()
        else:
            features[f'shank{g}_amp_mean'] = 0
            features[f'shank{g}_amp_max'] = 0
            features[f'shank{g}_amp_std'] = 0
            features[f'shank{g}_energy'] = 0
            features[f'shank{g}_dominant_ch'] = 0
            for ch in range(nCh):
                features[f'shank{g}_ch{ch}_amp'] = 0
    
    features['total_spikes'] = total_spikes
    features['length'] = len(row['groups'])
    
    idx = row['indexInDat']
    if len(idx) > 1:
        diffs = np.diff(idx.astype(float))
        features['isi_mean'] = diffs.mean()
        features['isi_std'] = diffs.std()
        features['isi_median'] = np.median(diffs)
        features['temporal_spread'] = idx[-1] - idx[0]
    else:
        features['isi_mean'] = 0
        features['isi_std'] = 0
        features['isi_median'] = 0
        features['temporal_spread'] = 0
    
    if total_spikes > 0:
        for g in range(nGroups):
            features[f'ratio_shank{g}'] = features[f'shank{g}_n_spikes'] / total_spikes
    else:
        for g in range(nGroups):
            features[f'ratio_shank{g}'] = 0
    
    return features

print('Extraction des features...')
features_list = []
for idx in range(len(df_moving)):
    if idx % 5000 == 0:
        print(f'  {idx}/{len(df_moving)}')
    features_list.append(extract_features(df_moving.iloc[idx], nGroups, nChannelsPerGroup))

features_df = pd.DataFrame(features_list).fillna(0)
print(f'Features : {features_df.shape}')

In [ ]:
# Préparer les targets
targets = np.array([[row['pos'][0], row['pos'][1]] for _, row in df_moving.iterrows()], dtype=np.float32)

# Normaliser les features
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
features_array = scaler.fit_transform(features_df.values).astype(np.float32)

print(f'Features: {features_array.shape}, Targets: {targets.shape}')

### 3b. Dataset séquentiel (fenêtres glissantes)

In [ ]:
CONTEXT_LEN = 10  # Nombre de fenêtres consécutives en entrée

class SequentialWindowDataset(Dataset):
    """
    Pour chaque index t, retourne les features des fenêtres [t-K+1, ..., t]
    et le target de la fenêtre t.
    """
    def __init__(self, features, targets, context_len=CONTEXT_LEN):
        self.features = features  # (N, D)
        self.targets = targets    # (N, 2)
        self.context_len = context_len
        self.n_samples = len(features) - context_len + 1
    
    def __len__(self):
        return self.n_samples
    
    def __getitem__(self, idx):
        # Séquence de features : [idx, idx+1, ..., idx+context_len-1]
        feat_seq = self.features[idx : idx + self.context_len]  # (context_len, D)
        target = self.targets[idx + self.context_len - 1]       # target de la dernière fenêtre
        return torch.from_numpy(feat_seq), torch.from_numpy(target)


# Split temporel
split_idx = int(len(features_array) * 0.8)

train_features = features_array[:split_idx]
train_targets = targets[:split_idx]
test_features = features_array[split_idx:]
test_targets = targets[split_idx:]

# Le scaler doit être fit seulement sur le train set
scaler_train = StandardScaler()
train_features = scaler_train.fit_transform(train_features).astype(np.float32)
test_features = scaler_train.transform(test_features).astype(np.float32)

train_dataset = SequentialWindowDataset(train_features, train_targets, CONTEXT_LEN)
test_dataset = SequentialWindowDataset(test_features, test_targets, CONTEXT_LEN)

BATCH_SIZE = 128
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=0)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=0)

print(f'Context length: {CONTEXT_LEN} fenêtres (~{CONTEXT_LEN * 108}ms)')
print(f'Feature dim: {train_features.shape[1]}')
print(f'Train: {len(train_dataset)} séquences, Test: {len(test_dataset)} séquences')

# Vérification
feat_seq, tgt = train_dataset[0]
print(f'\nBatch test: features={feat_seq.shape}, target={tgt.shape}')

### 3c. Architecture GRU

In [ ]:
class TemporalGRU(nn.Module):
    """
    GRU qui prend une séquence de vecteurs de features (une par fenêtre de 108ms)
    et prédit la position (x, y) de la dernière fenêtre.
    
    Architecture :
    - Projection linéaire des features vers dim cachée
    - GRU bidirectionnel (2 couches)
    - On prend le dernier hidden state → Dense → (x, y)
    """
    def __init__(self, input_dim, hidden_dim=128, num_layers=2, dropout=0.3, bidirectional=True):
        super().__init__()
        
        self.input_proj = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Dropout(dropout),
        )
        
        self.gru = nn.GRU(
            input_size=hidden_dim,
            hidden_size=hidden_dim,
            num_layers=num_layers,
            batch_first=True,
            dropout=dropout if num_layers > 1 else 0,
            bidirectional=bidirectional,
        )
        
        gru_output_dim = hidden_dim * (2 if bidirectional else 1)
        
        self.output_head = nn.Sequential(
            nn.Linear(gru_output_dim, hidden_dim),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim, 2),  # (x, y)
        )
    
    def forward(self, x):
        # x: (batch, context_len, input_dim)
        x = self.input_proj(x)          # (batch, context_len, hidden_dim)
        output, h_n = self.gru(x)       # output: (batch, context_len, gru_output_dim)
        last_output = output[:, -1, :]  # Dernier timestep (batch, gru_output_dim)
        return self.output_head(last_output)  # (batch, 2)


input_dim = train_features.shape[1]
model_gru = TemporalGRU(input_dim=input_dim, hidden_dim=128, num_layers=2, dropout=0.3)
n_params = sum(p.numel() for p in model_gru.parameters())
print(f'GRU : {n_params:,} paramètres')
print(model_gru)

# Test forward
test_input = torch.randn(4, CONTEXT_LEN, input_dim)
test_output = model_gru(test_input)
print(f'\nTest: input {test_input.shape} → output {test_output.shape}')

### 3d. Entraînement du GRU

In [ ]:
LR = 1e-3
WEIGHT_DECAY = 1e-3
EPOCHS = 50
PATIENCE = 10

model_gru = TemporalGRU(input_dim=input_dim, hidden_dim=128, num_layers=2, dropout=0.3).to(DEVICE)

optimizer = optim.AdamW(model_gru.parameters(), lr=LR, weight_decay=WEIGHT_DECAY)
scheduler = optim.lr_scheduler.OneCycleLR(
    optimizer, max_lr=LR, epochs=EPOCHS, steps_per_epoch=len(train_loader)
)
criterion = nn.MSELoss()

print(f'Entraînement GRU sur {DEVICE} pour {EPOCHS} epochs (patience={PATIENCE})')

best_val_loss = float('inf')
patience_counter = 0
train_losses = []
val_losses = []

for epoch in range(EPOCHS):
    # Train
    model_gru.train()
    epoch_loss = 0
    n_batches = 0
    for feat_seq, tgt in train_loader:
        feat_seq = feat_seq.to(DEVICE)
        tgt = tgt.to(DEVICE)
        
        optimizer.zero_grad()
        pred = model_gru(feat_seq)
        loss = criterion(pred, tgt)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model_gru.parameters(), 1.0)
        optimizer.step()
        scheduler.step()
        
        epoch_loss += loss.item()
        n_batches += 1
    
    train_loss = epoch_loss / n_batches
    
    # Eval
    model_gru.eval()
    val_loss_sum = 0
    val_batches = 0
    with torch.no_grad():
        for feat_seq, tgt in test_loader:
            feat_seq = feat_seq.to(DEVICE)
            tgt = tgt.to(DEVICE)
            pred = model_gru(feat_seq)
            val_loss_sum += criterion(pred, tgt).item()
            val_batches += 1
    
    val_loss = val_loss_sum / val_batches
    train_losses.append(train_loss)
    val_losses.append(val_loss)
    
    lr_current = optimizer.param_groups[0]['lr']
    print(f'Epoch {epoch+1:02d}/{EPOCHS} | Train: {train_loss:.5f} | Val: {val_loss:.5f} | LR: {lr_current:.6f}')
    
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        patience_counter = 0
        torch.save(model_gru.state_dict(), '../outputs/best_gru.pt')
    else:
        patience_counter += 1
        if patience_counter >= PATIENCE:
            print(f'Early stopping à epoch {epoch+1}')
            break

print(f'\nMeilleure val loss: {best_val_loss:.5f}')

In [ ]:
# Courbes d'entraînement
fig, ax = plt.subplots(figsize=(10, 5))
ax.plot(train_losses, label='Train', linewidth=2)
ax.plot(val_losses, label='Validation', linewidth=2)
ax.set_xlabel('Epoch')
ax.set_ylabel('MSE Loss')
ax.set_title('Courbes d\'entraînement - GRU multi-fenêtre')
ax.legend()
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

### 3e. Évaluation du GRU

In [ ]:
# Charger le meilleur modèle
model_gru.load_state_dict(torch.load('../outputs/best_gru.pt', map_location=DEVICE, weights_only=True))
model_gru.eval()

all_preds_gru = []
all_true_gru = []

with torch.no_grad():
    for feat_seq, tgt in test_loader:
        feat_seq = feat_seq.to(DEVICE)
        pred = model_gru(feat_seq)
        all_preds_gru.append(pred.cpu().numpy())
        all_true_gru.append(tgt.numpy())

y_pred_gru = np.concatenate(all_preds_gru)
y_true_gru = np.concatenate(all_true_gru)

res_gru = evaluate('GRU multi-fenêtre', y_true_gru, y_pred_gru)

# Appliquer Kalman vitesse
y_pred_gru_kalman = kalman_velocity(y_pred_gru, 0.001, 0.02, 0.02)
res_gru_kalman = evaluate('GRU + Kalman vitesse', y_true_gru, y_pred_gru_kalman)

# Sauvegarder
np.save('../outputs/preds_gru.npy', y_pred_gru)
np.save('../outputs/y_test_gru.npy', y_true_gru)
np.save('../outputs/preds_gru_kalman_vel.npy', y_pred_gru_kalman)
print('\nPrédictions GRU sauvegardées.')

In [ ]:
# Visualisations GRU
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
seg = slice(0, 500)

eucl_gru = res_gru['eucl']
eucl_gru_k = res_gru_kalman['eucl'] if 'eucl' in res_gru_kalman else np.sqrt(
    (y_true_gru[:, 0] - y_pred_gru_kalman[:, 0])**2 + (y_true_gru[:, 1] - y_pred_gru_kalman[:, 1])**2)

# Scatter
ax = axes[0, 0]
ax.scatter(y_true_gru[:, 0], y_pred_gru[:, 0], s=1, alpha=0.3)
ax.plot([0, 1], [0, 1], 'r--', linewidth=2)
ax.set_xlabel('True X'); ax.set_ylabel('Pred X')
ax.set_title(f'GRU - Position X (R²={res_gru["r2_x"]:.3f})')
ax.set_aspect('equal')

ax = axes[0, 1]
ax.scatter(y_true_gru[:, 1], y_pred_gru[:, 1], s=1, alpha=0.3)
ax.plot([0, 1], [0, 1], 'r--', linewidth=2)
ax.set_xlabel('True Y'); ax.set_ylabel('Pred Y')
ax.set_title(f'GRU - Position Y (R²={res_gru["r2_y"]:.3f})')
ax.set_aspect('equal')

# Trajectoire
ax = axes[1, 0]
ax.plot(y_true_gru[seg, 0], y_true_gru[seg, 1], 'b-', alpha=0.5, label='Vérité', linewidth=1.5)
ax.plot(y_pred_gru[seg, 0], y_pred_gru[seg, 1], 'r-', alpha=0.4, label='GRU brut', linewidth=1)
ax.plot(y_pred_gru_kalman[seg, 0], y_pred_gru_kalman[seg, 1], 'g-', alpha=0.7, label='GRU + Kalman vel', linewidth=1.5)
ax.set_xlabel('X'); ax.set_ylabel('Y')
ax.set_title('Trajectoire (500 points test)')
ax.legend(); ax.set_aspect('equal')

# CDF
ax = axes[1, 1]
for label, errors, color in [
    ('GRU brut', eucl_gru, 'red'),
    ('GRU + Kalman vel', eucl_gru_k, 'green'),
]:
    sorted_e = np.sort(errors)
    ax.plot(sorted_e, np.linspace(0, 1, len(sorted_e)), label=label, linewidth=2, color=color)
ax.set_xlabel('Erreur euclidienne'); ax.set_ylabel('CDF')
ax.set_title('CDF des erreurs')
ax.legend(); ax.grid(True, alpha=0.3)

plt.suptitle(f'GRU multi-fenêtre (contexte = {CONTEXT_LEN} × 108ms = {CONTEXT_LEN * 108}ms)',
             fontsize=14, fontweight='bold', y=1.01)
plt.tight_layout()
plt.show()

---
## Résumé et sauvegarde finale

In [ ]:
print('=== RÉSUMÉ DES AMÉLIORATIONS ===')
print()

# Partie 1 : Kalman vitesse
print('--- Partie 1 : Kalman vitesse constante ---')
for model_name in preds:
    y_true = y_tests[model_name]
    y_pred_brut = preds[model_name]
    y_pos = kalman_position(y_pred_brut, 0.001, 0.02)
    y_vel = kalman_velocity(y_pred_brut, 0.001, 0.02, 0.02)
    
    e_brut = np.sqrt((y_true[:, 0] - y_pred_brut[:, 0])**2 + (y_true[:, 1] - y_pred_brut[:, 1])**2).mean()
    e_pos = np.sqrt((y_true[:, 0] - y_pos[:, 0])**2 + (y_true[:, 1] - y_pos[:, 1])**2).mean()
    e_vel = np.sqrt((y_true[:, 0] - y_vel[:, 0])**2 + (y_true[:, 1] - y_vel[:, 1])**2).mean()
    
    print(f'  {model_name:15s} : brut={e_brut:.4f}, Kalman pos={e_pos:.4f}, Kalman vel={e_vel:.4f}')

print()
print('--- Partie 2 : Ensemble ---')
e_weighted = np.sqrt((y_true_ensemble[:, 0] - y_weighted[:, 0])**2 + (y_true_ensemble[:, 1] - y_weighted[:, 1])**2).mean()
e_ens_kalman = np.sqrt((y_true_ensemble[:, 0] - y_ensemble_kalman[:, 0])**2 + (y_true_ensemble[:, 1] - y_ensemble_kalman[:, 1])**2).mean()
print(f'  Moyenne pondérée           : {e_weighted:.4f}')
print(f'  Moyenne pondérée + Kalman  : {e_ens_kalman:.4f}')

print()
print('--- Partie 3 : GRU multi-fenêtre ---')
e_gru = res_gru['eucl'].mean()
e_gru_k = np.sqrt((y_true_gru[:, 0] - y_pred_gru_kalman[:, 0])**2 + (y_true_gru[:, 1] - y_pred_gru_kalman[:, 1])**2).mean()
print(f'  GRU brut                   : {e_gru:.4f}')
print(f'  GRU + Kalman vitesse       : {e_gru_k:.4f}')

print()
print('Fichiers sauvegardés :')
for f in ['../outputs/preds_xgboost_kalman_vel.npy', '../outputs/preds_transformer_kalman_vel.npy', '../outputs/preds_cnn_kalman_vel.npy',
          '../outputs/preds_ensemble_weighted.npy', '../outputs/preds_ensemble_kalman_vel.npy',
          '../outputs/preds_gru.npy', '../outputs/y_test_gru.npy', '../outputs/preds_gru_kalman_vel.npy',
          '../outputs/best_gru.pt']:
    exists = 'OK' if os.path.exists(f) else 'MANQUANT'
    print(f'  {f}: {exists}')

## Interprétation

### Kalman vitesse vs position

Le modèle de vitesse constante devrait mieux gérer les phases de mouvement rapide, car il **anticipe** la position suivante au lieu de simplement lisser. Le gain est surtout visible sur :
- Les virages (le modèle de position constante a du retard dans les virages)
- Les accélérations/décélérations

### Ensemble

La moyenne pondérée est simple mais efficace : chaque modèle contribue proportionnellement à sa fiabilité. Le stacking (Ridge) peut aller plus loin en apprenant des pondérations non-uniformes et des corrections de biais.

### GRU multi-fenêtre

C'est conceptuellement l'amélioration la plus profonde : le modèle **apprend** la continuité temporelle au lieu de la poser comme hypothèse (Kalman). Le GRU peut capturer des patterns complexes comme :
- Décélération à l'approche d'un mur
- Schémas de navigation récurrents (aller-retour dans le U-maze)
- Corrélation entre l'activité neuronale récente et la trajectoire future

### Limites

- Le GRU utilise les features XGBoost → il hérite de leurs limitations (pas de waveforms bruts)
- Le contexte de 10 fenêtres (~1 seconde) est arbitraire — on pourrait tester d'autres valeurs
- L'ensemble nécessite que tous les modèles aient le même test set et les mêmes indices

### Amélioration ultime : GRU sur embeddings Transformer

Pour aller plus loin, on pourrait extraire les embeddings du Transformer (le vecteur après masked average pooling, dim=64) pour chaque fenêtre, puis les passer au GRU. Cela combinerait la richesse des waveforms bruts (Transformer) avec la continuité temporelle (GRU).